In [1]:
import torch
import torch.autograd as autograd
from models.modules import *

ModuleNotFoundError: No module named 'modules'

In [2]:
models.modules

ModuleNotFoundError: No module named 'modules'

## Test 1

In [6]:
x = torch.randn(3, 3, 8, 8)
conv1 = Conv2d(3, 8, kernel_size=3, stride=1, padding=1, grad_proj=True)
conv2 = Conv2d(8, 8, kernel_size=3, stride=2, padding=1, grad_proj=True)
relu = ReLU(inplace=True, grad_proj=True)
pool = AvgPool2d(kernel_size=4, grad_proj=True)
fc = Linear(8, 10, grad_proj=True)

# forward pass
y, jvp = pool(*relu(*conv2(*relu(*conv1(x)))))
y, jvp = fc(torch.flatten(y, 1), torch.flatten(jvp, 1))

# backward pass
grad = autograd.grad(y[0, 0], 
  list(conv1.parameters()) 
  + list(conv2.parameters()) 
  + list(fc.parameters()))

# parameter gradients
conv1_weight_grad, conv1_bias_grad = grad[0], grad[1]
conv2_weight_grad, conv2_bias_grad = grad[2], grad[3]
fc_weight_grad, fc_bias_grad = grad[4], grad[5]

# random vectors
conv1_weight_rv, conv1_bias_rv = conv1.weight_rv, conv1.bias_rv
conv2_weight_rv, conv2_bias_rv = conv2.weight_rv, conv2.bias_rv
fc_weight_rv, fc_bias_rv = fc.weight_rv, fc.bias_rv

# brute-force calculation of Jacobian-vector product
true_jvp = (conv1_weight_grad * conv1_weight_rv).sum() \
         + (conv1_bias_grad * conv1_bias_rv).sum() \
         + (conv2_weight_grad * conv2_weight_rv).sum() \
         + (conv2_bias_grad * conv2_bias_rv).sum() \
         + (fc_weight_grad * fc_weight_rv).sum() \
         + (fc_bias_grad * fc_bias_rv).sum()

if abs(jvp[0, 0] - true_jvp) < 1e-5:
    print('TEST 1 PASSED')
else:
    print('TEST 1 FAILED')
    print('ground truth: %.3f 	 result: %.3f' % (true_jvp, jvp[0, 0]))
    exit()

NameError: name 'Conv2d' is not defined

In [2]:
jvp

NameError: name 'jvp' is not defined

## Number of params in last layer

In [95]:
import torchvision.models as pretrained_models
import numpy as np
import pandas as pd

In [81]:
resnet50 = pretrained_models.resnet50()
resnet18 = pretrained_models.resnet18()

In [82]:
# returns the number of parameters in the model or a part of the model
def num_parameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    return sum([np.prod(p.size()) for p in model_parameters]) 

In [87]:
num_parameters(resnet50.layer4) # last layer of resnet 50

14964736

In [88]:
num_parameters(resnet18.layer4) # last layer of resnet 18

8393728

# Johnson-Lindenstrauss bound for p (number of random projections)

This does not depend on the input dimension, but only on the batch size. Assuming we 

In [4]:
# This finds a ‘safe’ number of components to randomly project to; default eps=0.1
from sklearn.random_projection import johnson_lindenstrauss_min_dim

min_dim_dict = {}
bs = [2**i for i in range(10)]
for i in bs:
    min_dim_dict[i] = johnson_lindenstrauss_min_dim(i, eps=0.10)

# bs: safe number of dimensions
min_dim_dict

{1: 0,
 2: 594,
 4: 1188,
 8: 1782,
 16: 2376,
 32: 2970,
 64: 3564,
 128: 4158,
 256: 4753,
 512: 5347}

# Pseudo Code

In [136]:
# Assume the models as follows
from models.modules import *

t_net = resnet_rp.resnet50()
s_net = pretrained_models.resnet18()

batch_size = 32
n_classes = 1000 # imagenet

# Random input data
data = torch.randn(batch_size, 3, 8, 8)
labels = torch.randint(0, n_classes, (1000, 1))

NameError: name 'resnet18_rp' is not defined